In [24]:
!pip install regex

In [9]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd
import re
from recipe_scrapers import scrape_me
import heapq

#### GET SHEET #######
gc = gspread.authorize(GoogleCredentials.get_application_default()) #gc object with authentication to access
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1212czX2ZSBzMW81lqnGe0WvkS8iU3sTtS1CQXCeDI44/edit#gid=0') #get file
sheet = wb.sheet1 #get sheet

data = sheet.get_all_values()
df = pd.DataFrame(data) #turn matrix into data frame
df.columns = df.iloc[0] #make columns using row 1 as titles
df = df.iloc[1:]

# GENERATE TITLE AND BASIC INGREDIENTS ####
for index, row in df.iterrows():
    to_scrape = scrape_me(row['Link'])
    row['Title'] = to_scrape.title()
    row['Ingredients'] = to_scrape.ingredients()

set_with_dataframe(sheet, df)

# CLEAN FRACTIONS
for i in range(len(df)):
  ilist =  df.iloc[i]['Ingredients']
  for i in range(len(ilist)):
    # if re.match(r'[\u00BC-\u00BE\u2150-\u215E]', i):
    ilist[i] = re.sub(r'[\u00BC]', '1/4', ilist[i])
    ilist[i] = re.sub(r'[\u00BD]', '1/2', ilist[i])
    ilist[i] = re.sub(r'[\u00BE]', '3/4', ilist[i])
    ilist[i] = re.sub(r'[\u2150]', '1/7', ilist[i])
    ilist[i] = re.sub(r'[\u2151]', '1/9', ilist[i])
    ilist[i] = re.sub(r'[\u2152]', '1/10', ilist[i])
    ilist[i] = re.sub(r'[\u2153]', '1/3', ilist[i])
    ilist[i] = re.sub(r'[\u2154]', '2/3', ilist[i])
    ilist[i] = re.sub(r'[\u2155]', '1/5', ilist[i])
    ilist[i] = re.sub(r'[\u2156]', '2/5', ilist[i])
    ilist[i] = re.sub(r'[\u2157]', '3/5', ilist[i])
    ilist[i] = re.sub(r'[\u2158]', '4/5', ilist[i])
    ilist[i] = re.sub(r'[\u2159]', '1/6', ilist[i])
    ilist[i] = re.sub(r'[\u215A]', '5/6', ilist[i])
    ilist[i] = re.sub(r'[\u215B]', '1/8', ilist[i])
    ilist[i] = re.sub(r'[\u215C]', '3/8', ilist[i])
    ilist[i] = re.sub(r'[\u215D]', '5/8', ilist[i])
    ilist[i] = re.sub(r'[\u215E]', '7/8', ilist[i])

set_with_dataframe(sheet, df)

# MAKE DICTIONARY
df = df.dropna()
for i in range(len(df)): #for each ingredients list
  ilist = df.iloc[i]['Ingredients']
  ingredients = [None]*len(ilist)
  keys = []
  values = []

  for a in range(len(ilist)): #create keys/values
    measures = re.match(r'(.*\b(?:teaspoon|teaspoons|tablespoon|tablespoons|cups|cup|can|dash|pinch)\b)(\s)(.*)', ilist[a])
    no_meas = re.match(r'(\d)(\s)(.*)', ilist[a])
    if measures:
      keys.append(measures.group(1))
      values.append(measures.group(3))
    elif bool(measures) == False and bool(no_meas) == True:
      keys.append(no_meas.group(1))
      values.append(no_meas.group(3))
    else:
      keys.append('0')
      values.append(ilist[a])

  for d in range(len(ilist)): #compile into ingredients dictionary
    ingredients[d] = [keys[d], values[d]]

#HOW TO GET INGREDIENTS INTO DATA FRAME??
  df.iloc[i]['Ingredients'] = ingredients

df.head()
set_with_dataframe(sheet, df)

##########################################################

my_items = ['butter', 'milk', 'egg']
ret = []

for c in range(len(df)):
  link = df.iloc[c]['Link']
  ingredients = scrape_me(link).ingredients()
  ing = ' '.join(ingredients) #string of all ingredients

  
  for g in my_items:
    if re.match(f'{g}', ing):
      ret.append([df.iloc[c]['Title'], 1])
  
print(ret)
                 



# for c in range(len(df)): #go down rows
#   ingredients = df.iloc[c]['Ingredients']
#   # print(ingredients)
#   for itempair in ingredients: #for each itempair in a recipe
#     item = itempair[1]
#     if re.match(f'{my_items[0]}', item): #if item matches..
#       return_dict[df.iloc[c]['Title']] = 1 #put a count on it

# print(return_dict)

# for c in range(len(df)):
#   if df.iloc[c]['Title'] in return_dict.keys():
#     ingredients = df.iloc[c]['Ingredients']
#     for itempair in ingredients: #for each itempair in a recipe
#       item = itempair[1]
#       if re.match(f'{my_items[1]}', item): #if item matches..
#         return_dict[df.iloc[c]['Title']] = 2 #put a count on it

# print(return_dict)







[]
